# 过滤与查看 pandas 表格中的局部数据

这一篇来学习DataFrame 强大的数据查询与过滤能力。首先准备数据：

In [2]:
import pandas as pd

# 将列索引保存在 index_arr 变量中
index_arr = ["姓名", "年龄", "籍贯", "部门"]
# 构建小明、小亮、小E的行 Series，并使用我们创建好的 index_arr 作为 Series 的index
ser_xm = pd.Series(["小明", 22, "河北", "IT部"], index=index_arr)
ser_xl = pd.Series(["小亮", 25, "广东", "IT部"], index=index_arr)
ser_xe = pd.Series(["小E", 23, "四川", "财务部"], index=index_arr)
ser_xli = pd.Series(["小李", 22, "云南", "设计部"], index=index_arr)
ser_xw = pd.Series(["小王", 20, "福建", "设计部"], index=index_arr)
# 直接将三个 Series 以列表的形式作为 DataFrame 的参数，创建 DataFrame
df_info = pd.DataFrame([ser_xm, ser_xl, ser_xe, ser_xli, ser_xw])
# 使用 notebook 打印 DataFrame
df_info

,姓名,年龄,籍贯,部门
0,小明,22,河北,IT部
1,小亮,25,广东,IT部
2,小E,23,四川,财务部
3,小李,22,云南,设计部
4,小王,20,福建,设计部


## 1 使用 `[]` 查询元素

In [3]:
# 对 DataFrame 使用 [] ，返回列名 = 传入的索引值的 Series
ser_name = df_info["姓名"]
ser_name

0    小明
1    小亮
2    小E
3    小李
4    小王
Name: 姓名, dtype: object

In [4]:
# 对 Series 使用 [] ，取回索引值
name_of_xl = ser_name[1]
name_of_xl

'小亮'


## 2 多重索引

`[]` 除了可以传入单一索引实现数据选择，还支持传入一个**索引列表**来获得原始数据集的一个子集。规则如下：

- 当对 DataFrame 的 `[]` 索引器传入一个索引列表时，返回一个新的 DataFrame，这个 DataFrame 只包含索引列表中的列，相当于是原 DataFrame 的子 DataFrame；
- 同理对于 Series，传入索引列表时，返回一个子 Series，包含索引列表对应的数据。

In [5]:
# 返回一个新的 DataFrame
sub_df_info = df_info[["姓名", "年龄"]]
sub_df_info

,姓名,年龄
0,小明,22
1,小亮,25
2,小E,23
3,小李,22
4,小王,20


In [6]:
# 返回一个新的 Series
sub_ser_name = ser_name[[0, 2]]
sub_ser_name

0    小明
2    小E
Name: 姓名, dtype: object

## 3 范围选择

In [7]:
# df[n:m], 选择第 n 条到第 m 条之间的记录。
df_info[2:4]

,姓名,年龄,籍贯,部门
2,小E,23,四川,财务部
3,小李,22,云南,设计部


In [8]:
# df[:m], 选择前 m 条记录。示例代码：
df_info[:2]

,姓名,年龄,籍贯,部门
0,小明,22,河北,IT部
1,小亮,25,广东,IT部


In [9]:
# df[::n], 间隔 n 条选择。
df_info[::2]

,姓名,年龄,籍贯,部门
0,小明,22,河北,IT部
2,小E,23,四川,财务部
4,小王,20,福建,设计部


In [10]:
# df[::-1], 从最后一条开始逐一选择。
df_info[::-1]

,姓名,年龄,籍贯,部门
4,小王,20,福建,设计部
3,小李,22,云南,设计部
2,小E,23,四川,财务部
1,小亮,25,广东,IT部
0,小明,22,河北,IT部


## 4 `loc` 与 `iloc`

上面，如果我们想查询 DataFrame 中某个单元格的数据，那往往都需要分三步走：

1. 查看单元格所在行的索引；
2. 拿到单元格所在列的列 Series；
3. 用步骤 1 拿到的索引去步骤 2 拿到的 Series 中查询出具体的数据。

这种方式比较繁琐，而 pandas 除了 `[]` 索引器之外，还提供了一套非常强大的数据查询方式：`loc` 和 `iloc`。

loc 和 iloc 是 pandas 的一种特殊的对象，专门用于查询 DataFrame 中的数据。

- loc 的用法：`df.loc[行索引名称, 列索引名称]`；
- iloc 与 loc 的用法相同，但是只能传入整数索引。

In [11]:
# loc 示例
print("小亮的部门：", df_info.loc[1, "部门"])

print("-------------------------------")

# loc 对象的 [] 索引器支持所有 DataFrame 的 [] 索引器的能力。具体来说就是 loc 对象的行索引部分和列索引部分都可以分别上面介绍的多种索引、范围选择的语法。

print(df_info.loc[::-1,["姓名","年龄"]])

print("-------------------------------")

# 从输出来看，效果和使用 loc 对象是一样的。但是从易用性的层面，loc 显然比 iloc 更加容易使用且不容易出错。使用 iloc 每次都需要去数我们所要的数据在第几行、第几列，非常容易出错。所以一般情况下，我们都推荐直接使用 loc。但也有一些场景，不知道行索引，但明确要拿第一个元素的场合就需要使用 iloc。
# 总体来说 iloc 是和 loc 打配合使用的， loc 最常用。
print("小亮的部门：", df_info.iloc[1, 3])

小亮的部门： IT部
-------------------------------
   姓名  年龄
4  小王  20
3  小李  22
2  小E  23
1  小亮  25
0  小明  22
-------------------------------
小亮的部门： IT部


## 5 条件查询

loc 的条件索引的具体用法如下所示，它和普通的 loc 用法区别最大的就是将行索引部分替换为条件表达式。

```python
df.loc[条件表达式, 列索引名称]
```

In [12]:
# 条件表达式：df_info["年龄"] >= 23， 然后因为我们希望获得所有列，所以列索引直接写:
age_parted_df = df_info.loc[df_info["年龄"] >= 23,:]
print(type(age_parted_df))
age_parted_df

<class 'pandas.core.frame.DataFrame'>


,姓名,年龄,籍贯,部门
1,小亮,25,广东,IT部
2,小E,23,四川,财务部


In [13]:
# 使用条件表达式: df["姓名"] == "小亮", 并取 部门 这个列
department_of_xl = df_info.loc[df_info["姓名"] == "小亮", "部门"]
print(type(department_of_xl))
department_of_xl

<class 'pandas.core.series.Series'>


1    IT部
Name: 部门, dtype: object

In [14]:
# iloc[0] 的意思就是第 0 行
print("小亮的部门：" + df_info.loc[df_info["姓名"] == "小亮", "部门"].iloc[0])

小亮的部门：IT部


## 6 高级条件查询

当我们筛选数据的时候，一个条件不能满足要求，就需要组合多个条件来筛选出我们想要的数据。组合多个条件时，最常见的两个逻辑关系就是：逻辑与和逻辑或。

In [15]:
# loc 条件表达式的逻辑与操作。每个子条件都需要用括号括起来
df_info.loc[(df_info["部门"] == "IT部") & (df_info["年龄"] <25), :]

,姓名,年龄,籍贯,部门
0,小明,22,河北,IT部


In [16]:
df_info.loc[(df_info["部门"] == "财务部") | (df_info["部门"] == "设计部"), :]

,姓名,年龄,籍贯,部门
2,小E,23,四川,财务部
3,小李,22,云南,设计部
4,小王,20,福建,设计部
